In [16]:
import time
import torch
from torch.distributions import Categorical, kl
import numpy as np
from tqdm.auto import tqdm
# from d2l.torch import Animator

from net_deepaco import Net as Net_DeepACO
from net import Net
from aco import ACO
from faco import FACO
from utils import gen_pyg_data, load_test_dataset

torch.manual_seed(12345)

EPS = 1e-10
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f'Using device: {device}')

Using device: cuda:0


In [17]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
%matplotlib inline

In [18]:
from xml.parsers.expat import model

from copy import copy
from train import attach_faco_state_to_pyg
from utils import gen_pyg_data, load_test_dataset


MIN_NEW_EDGES = 8
K_NEAREST = 20
SAMPLE_TWO_OPT = True
INV_TEMP = 1.0
T_eval = 5

@torch.no_grad()
def infer_instance(model, demands, distances, n_ants, t_aco_diff, visualize=False):
    model.eval()
    pyg_base = gen_pyg_data(demands, distances, device)
    results = torch.zeros(size=(len(t_aco_diff),), device=device)

    solver = FACO(
        distances=distances,
        demand=demands,
        n_ants=n_ants,
        k_nearest=K_NEAREST,
        capacity=50,
        decay=0.9,
        alpha=1.0,
        beta=1.0,
        min_new_edges=MIN_NEW_EDGES,
        sample_two_opt=SAMPLE_TWO_OPT,
        device=device,
    )

    if visualize:
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        fig.suptitle('Heuristic Matrix Evolution')
        cbar1, cbar2 = None, None

    # baseline (first step sampling mean)
    ref_flat = solver.best_flat.copy()
    tau = solver.pheromone_sparse.detach().clone()
    pyg = copy(pyg_base)
    pyg = attach_faco_state_to_pyg(pyg, solver, ref_flat, tau, net=model)
    out = model(pyg)
    heu_vec = out
    heu_full = model.reshape(pyg, heu_vec) + 1e-10
    solver.set_heuristic(heu_full)

    costs_np, flats, _touched = solver.sample(invtemp=INV_TEMP, require_prob=False)

    T_eval = t_aco_diff[-1]
    for _t in range(T_eval):
        ref_flat = solver.best_flat.copy()
        tau = solver.pheromone_sparse.detach().clone()

        pyg = copy(pyg_base)
        pyg = attach_faco_state_to_pyg(pyg, solver, ref_flat, tau, net=model)

        out = model(pyg)
        heu_vec = out
        heu_full = model.reshape(pyg, heu_vec) + 1e-10
        solver.set_heuristic(heu_full)

        if visualize:
            # Remove old colorbars if they exist
            if cbar1 is not None:
                cbar1.remove()
            if cbar2 is not None:
                cbar2.remove()
            
            # Clear previous plots
            for ax in axes:
                ax.clear()
            
            # Plot heuristic matrix
            heu_cpu = heu_full.cpu().numpy()
            im1 = axes[0].imshow(heu_cpu, cmap='viridis', aspect='auto')
            axes[0].set_title(f'Heuristic Matrix (Iteration {_t+1})')
            axes[0].set_xlabel('To Node')
            axes[0].set_ylabel('From Node')
            cbar1 = plt.colorbar(im1, ax=axes[0])
            
            # Plot pheromone matrix (convert sparse to dense for visualization)
            tau_dense = tau.to_dense().cpu().numpy() if hasattr(tau, 'to_dense') else tau.cpu().numpy()
            im2 = axes[1].imshow(tau_dense, cmap='hot', aspect='auto')
            axes[1].set_title(f'Pheromone Matrix (Iteration {_t+1})')
            axes[1].set_xlabel('To Node')
            axes[1].set_ylabel('From Node')
            cbar2 = plt.colorbar(im2, ax=axes[1])
            
            # Update display
            clear_output(wait=True)
            display(fig)
            plt.pause(0.1)

        costs_np, flats, _touched = solver.sample(invtemp=INV_TEMP, require_prob=False)
        best_i = int(np.argmin(costs_np))
        best_cost = float(costs_np[best_i])
        best_flat = flats[best_i]

        if best_cost < solver.best_cost:
            solver.best_cost = best_cost
            solver.best_flat = best_flat
        solver._update_pheromone_from_flat(best_flat, best_cost)

        # Check if (_t + 1) is in t_aco_diff and get its index
        if (_t + 1) in t_aco_diff:
            i = t_aco_diff.index(_t + 1)
            results[i] = solver.best_cost

    if visualize:
        plt.close(fig)
    
    return results
        
@torch.no_grad()
def infer_instance_deepaco(model, demands, distances, n_ants, t_aco_diff):
    if model:
        model.eval()
        pyg_data = gen_pyg_data(demands, distances, device)
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
        aco = ACO(
            distances=distances,
            demand=demands,
            n_ants=n_ants,
            heuristic=heu_mat,
            device=device
        )
    else:
        aco = ACO(
            distances=distances,
            demand=demands,
            n_ants=n_ants,
            device=device
        )
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t)
        results[i] = best_cost
    return results
        

@torch.no_grad()
def test(dataset, model, n_ants, t_aco, visualize=False):
    # _t_aco = [0] + t_aco
    # t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco),), device=device)
    bar = tqdm(dataset, dynamic_ncols=True)
    start = time.time()
    for idx, (demands, distances) in enumerate(bar):
        # Only visualize the first instance if requested
        viz = visualize and idx == 0
        results = infer_instance(model, demands, distances, n_ants, t_aco, visualize=viz)
        sum_results += results
        bar.set_description(f'Avg costs: {(sum_results/(bar.n)).cpu().numpy()}')
    end = time.time()
    
    return sum_results / len(dataset), end-start

@torch.no_grad()
def test_deepaco(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    bar = tqdm(dataset, dynamic_ncols=True)
    start = time.time()
    for demands, distances in bar:
        results = infer_instance_deepaco(model, demands, distances, n_ants, t_aco_diff)
        sum_results += results
        bar.set_description(f'Avg costs: {(sum_results/(bar.n)).cpu().numpy()}')
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on TSP20

In [19]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)

net = Net(value_head=True).to(device)
net.load_state_dict(torch.load(f'../pretrained/cvrp/faco_cvrp{n_node}_ppo_nstep.pt', map_location=device))
avg_aco_best, duration = test(test_list, net, n_ants, t_aco, visualize=False)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i])) 
  

  0%|          | 0/100 [00:00<?, ?it/s]

total duration:  72.1379771232605
T=1, average obj. is 5.210780143737793.
T=10, average obj. is 4.714725017547607.
T=50, average obj. is 4.646577835083008.
T=100, average obj. is 4.6398162841796875.


DeepACO

In [20]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)

net = Net_DeepACO().to(device)
net.load_state_dict(torch.load(f'../pretrained/cvrp/cvrp{n_node}.pt', map_location=device))
avg_aco_best, duration = test_deepaco(test_list, net, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i])) 
  

  0%|          | 0/100 [00:00<?, ?it/s]

total duration:  120.70868515968323
T=1, average obj. is 5.081987380981445.
T=10, average obj. is 4.84982967376709.
T=50, average obj. is 4.803032398223877.
T=100, average obj. is 4.793013095855713.


NGFACO

ACO

In [21]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)

net = Net(value_head=True).to(device)
net.load_state_dict(torch.load(f'../pretrained/cvrp/faco_cvrp{n_node}_ppo_nstep.pt', map_location=device))
avg_aco_best, duration = test(test_list, net, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i])) 
  

  0%|          | 0/100 [00:00<?, ?it/s]

total duration:  71.1493182182312
T=1, average obj. is 5.239306449890137.
T=10, average obj. is 4.719290733337402.
T=50, average obj. is 4.617067813873291.
T=100, average obj. is 4.609489917755127.


In [22]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)


avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

  0%|          | 0/100 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'eval'

### Test with Visualization (Single Instance)

In [ ]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 50, 100]
test_list = load_test_dataset(n_node, device)

# Load model
net = Net(value_head=True).to(device)
net.load_state_dict(torch.load(f'../pretrained/cvrp/faco_cvrp{n_node}_ppo_nstep.pt', map_location=device))

# Test with visualization enabled (will only visualize first instance)
avg_aco_best, duration = test(test_list[:1], net, n_ants, t_aco, visualize=True)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {:.2f}.".format(t, avg_aco_best[i]))

NameError: name 'load_test_dataset' is not defined